# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [25]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [29]:
#import nltk
#nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/carrie/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
df['combined_text'] = df['headline'] + ' ' +  df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [31]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [32]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [33]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55566537, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [34]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [35]:
wv.most_similar('Texas')

[('Louisiana', 0.8332939743995667),
 ('Oregon', 0.8179298639297485),
 ('Pennsylvania', 0.8092736005783081),
 ('Illinois', 0.8029823303222656),
 ('Georgia', 0.7991378307342529),
 ('Florida', 0.7938148975372314),
 ('Arkansas', 0.7872060537338257),
 ('Wisconsin', 0.7835549712181091),
 ('Arizona', 0.7766444087028503),
 ('Maryland', 0.7763110399246216)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [36]:
wv.most_similar(negative='Texas')

[('double-dipping', 0.3829522132873535),
 ('Un-Blind', 0.3814493417739868),
 ('Drifters', 0.3789266347885132),
 ('Foster-To-Adopt', 0.3760555386543274),
 ('Unabashedly', 0.37529247999191284),
 ('Parent/Grandparent', 0.37517061829566956),
 ('exiling', 0.37288331985473633),
 ('once-reliable', 0.366895854473114),
 ('Begets', 0.3664758801460266),
 ('Anti-Press', 0.36385655403137207)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [37]:
wv['Texas']

array([ 1.0477536e+00,  1.1778516e+00,  1.8327588e+00,  9.8673103e-04,
        5.6205195e-01, -1.7291804e-01, -4.8628879e-01,  8.3395737e-01,
        3.9488018e+00,  3.9012250e-01, -1.5239400e+00, -8.1849563e-01,
       -4.5142809e-01, -3.3880654e-01,  3.2240093e+00, -6.8819571e-01,
        2.9641172e-01,  1.4658880e+00, -1.9436227e+00, -4.8649135e-01,
        1.7945235e+00,  3.1960812e+00,  1.8875937e+00, -1.5556654e+00,
       -8.8539600e-01,  5.0520366e-01, -8.4324598e-01,  1.4021816e+00,
       -3.2244756e+00,  2.7464724e+00,  1.0102137e+00, -2.3746996e+00,
       -1.2675080e+00,  3.2560644e-01,  1.6916499e+00,  4.1030586e-01,
       -9.5999770e-02,  4.5646477e-01,  8.1076825e-01, -3.7627926e-01,
        4.9337226e-01,  4.8755221e-02,  7.1831167e-01, -1.6119236e+00,
       -2.7005193e+00,  1.3561854e+00,  4.0027100e-01, -9.8002434e-01,
        2.0502827e-01,  1.8550968e-01,  8.4295917e-01,  2.6012677e-01,
        1.0172832e+00, -3.4591794e-01, -1.9759357e-01,  3.7881175e-01,
      

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [38]:
wv.vectors

array([[-1.598379  , -0.49303558,  1.2774211 , ...,  0.82848513,
        -1.145846  ,  0.8276065 ],
       [-0.4339047 ,  0.6596146 , -0.32893354, ...,  2.2875268 ,
        -0.44392288, -0.20079543],
       [-0.29943386,  0.3779975 ,  1.1926714 , ..., -0.07800385,
        -0.87595785,  0.64032954],
       ...,
       [-0.06739926, -0.02840828,  0.0962557 , ..., -0.04556446,
         0.03415365, -0.04509638],
       [-0.07332389, -0.04270183,  0.08092176, ...,  0.10502681,
        -0.0365148 , -0.00340179],
       [-0.01539152, -0.0303794 , -0.01912877, ..., -0.01523098,
         0.03576544, -0.0409952 ]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [39]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('brunette', 0.6037125587463379),
 ('princess', 0.5998502969741821),
 ('queen', 0.5929369330406189),
 ('crown', 0.5913313627243042),
 ('title', 0.5853015780448914),
 ('revival', 0.5837910771369934),
 ('symbol', 0.551271378993988),
 ('Queen', 0.5416910648345947),
 ('villain', 0.5389100313186646),
 ('monster', 0.5385596752166748)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!